In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
from random import randint
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import time as t
import pandas as pd
from selenium.webdriver.chrome.options import Options
import csv

In [2]:
base_url = 'https://www.in-cosmetics.com/global/en-gb/exhibitor-directory.html#/'
driver = webdriver.Chrome()
# driver.maximize_window()
driver.get(base_url)

# Wait for the cookie banner to be present
cookie_banner = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, 'onetrust-banner-sdk'))
)

# Find the "Accept All Cookies" button within the cookie banner
accept_all_button = cookie_banner.find_element(By.ID, 'onetrust-accept-btn-handler')

# Click the "Accept All Cookies" button
accept_all_button.click()
print('Accepted cookies')

Accepted cookies


In [3]:
# Scroll to the bottom of the page
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


In [4]:
#time.sleep(300) 

html_content = driver.page_source
driver.quit()
soup = BeautifulSoup(html_content, "html.parser")

In [5]:
# Find all the exhibitor name and URL pairs
exhibitor_links = soup.select('a[data-dtm="exhibitorDirectory_exhibitorName_bronze"]')

# Create a CSV file and write the headers
with open('exhibitors.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Name', 'URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Write each exhibitor to the CSV file
    for link in exhibitor_links:
        name = link.find('h3', class_='text-center-mobile wrap-word').text.strip()
        url = link['href']
        writer.writerow({'Name': name, 'URL': url})

print('All names in exhibitors.csv')

All names in exhibitors.csv


In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the webdriver
driver = webdriver.Chrome()

# Open the CSV file and loop over each row
with open('exhibitors.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    first_iteration = True  # Flag to track the first iteration

    for row in reader:
        url = row['URL']

        # Open the URL
        driver.get(url)

        # Wait for the cookie banner to be present and click the "Accept All Cookies" button (only for the first iteration)
        if first_iteration:
            try:
                cookie_banner = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.ID, 'onetrust-banner-sdk'))
                )
                accept_all_button = cookie_banner.find_element(By.ID, 'onetrust-accept-btn-handler')
                accept_all_button.click()
                first_iteration = False  # Set the flag to False after the first iteration
            except:
                print(f"Unable to click the 'Accept All Cookies' button for {url}")

        # Get the page source HTML
        html = driver.page_source

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Extract the company website, email, and phone number
        website_div = soup.find('div', {'id': 'exhibitor_details_website'})
        if website_div:
            website = website_div.find('a')['href']
        else:
            website = ''

        email_div = soup.find('div', {'id': 'exhibitor_details_email'})
        if email_div:
            email = email_div.find('a')['href'].replace('mailto:', '')
        else:
            email = ''

        phone_div = soup.find('div', {'id': 'exhibitor_details_phone'})
        if phone_div:
            phone = phone_div.find('a')['href'].replace('tel:', '')
        else:
            phone = ''

        # Update the row in the CSV file with the extracted information
        row['Website'] = website
        row['Email'] = email
        row['Phone'] = phone

        # Write the updated row back to the CSV file
        with open('exhibitors_full.csv', 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Name', 'URL', 'Website', 'Email', 'Phone']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)
        time.sleep(2)
        print(f"Processed: {row['Name']}")

# Close the webdriver
driver.quit()
print('All exhibitors processed successfully.')

Processed: 3V Sigma S.p.a.
Processed: A&A Fratelli Parodi
Processed: A&Z Science and Technology Ltd
Processed: A-Sense sp. z o.o.
Processed: AAK
Processed: Aako BV
Processed: Aarti Surfactants Limited
Processed: ABC NanoTech Co. Ltd.
Processed: ABC TEXTURE
Processed: Abich - Lifeanalytics
Processed: ABIHPEC*
Processed: ABIO MATERIALS
Processed: Actylis
Processed: ACS Phyto
Processed: Actalia Sensoriel
Processed: Actichem
Processed: Actifs Précieux
Processed: Active Concepts S.r.l
Processed: Active Micro Technologies
Processed: Active Peptide Company
Processed: Activen SA
Processed: Activon. Co. Ltd.
Processed: ADARA
Processed: ADEKA Europe Gmbh
Processed: Advanced Development & Safety Laboratories Ltd ADSL
Processed: Advanced Dispersed Particles SL
Processed: Advanced Organic Materials, S.A. - (AOM)
Processed: AE CHEMIE, INC.
Processed: AEGIS BEAUTY SOLUTIONS LIMITED
Processed: Aethera Biotech Srl
Processed: AGBV Nutrition GmbH
Processed: AGC Si-Tech Co.,Ltd.
Processed: Agrana Starke G

In [ ]:
# # now clean email column
# unique_links = set()
# for link in links:
#     clean_link = link.split("?")[0]
#     if clean_link.endswith("/"):
#         clean_link = clean_link[:-1]
#     unique_links.add(clean_link)
# unique_links = list(unique_links)



In [10]:
def find_linkedin_link(webpage):
    driver = webdriver.Chrome()
    driver.get(webpage)

    # Wait for the page to load
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Find the button containing "All" in its ID within the page
    accept_all_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button[id*="All"]')))

    # Click the "Accept All Cookies" button
    accept_all_button.click()

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    linkedin_links = set()
    all_links = soup.find_all("a")
    for a in all_links:
        if a.has_attr("href"):
            if "https://www.linkedin.com/company/" in a["href"]:
                clean_link = a["href"].split("?")[0]
                linkedin_links.add(clean_link)
    driver.quit()
    return list(linkedin_links)

In [11]:
website = 'http://www.fratelliparodi.it'
find_linkedin_link(website)

NoSuchFrameException: Message: no such frame
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00007FF6D655AD02+56930]
	(No symbol) [0x00007FF6D64CF602]
	(No symbol) [0x00007FF6D638419D]
	(No symbol) [0x00007FF6D638CD06]
	(No symbol) [0x00007FF6D6409D1A]
	(No symbol) [0x00007FF6D63EBC9A]
	(No symbol) [0x00007FF6D64081E2]
	(No symbol) [0x00007FF6D63EBA43]
	(No symbol) [0x00007FF6D63BD438]
	(No symbol) [0x00007FF6D63BE4D1]
	GetHandleVerifier [0x00007FF6D68D6F8D+3711213]
	GetHandleVerifier [0x00007FF6D69304CD+4077101]
	GetHandleVerifier [0x00007FF6D692865F+4044735]
	GetHandleVerifier [0x00007FF6D65F9736+706710]
	(No symbol) [0x00007FF6D64DB8DF]
	(No symbol) [0x00007FF6D64D6AC4]
	(No symbol) [0x00007FF6D64D6C1C]
	(No symbol) [0x00007FF6D64C68D4]
	BaseThreadInitThunk [0x00007FFE6ADD257D+29]
	RtlUserThreadStart [0x00007FFE6B8CAA58+40]


In [10]:
# Read the CSV file
df = pd.read_csv('exhibitors_full.csv', header=None, names=['Name', 'URL', 'Website', 'Email', 'Phone'])

# Create a new column for LinkedIn links
df['LinkedIn_link'] = ''

# Configure Chrome options to launch the driver in a minimized state
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--window-size=1920x1080")  # Set window size

# Open the output CSV file
with open('exhibitors_with_linkedin.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row with the new column name
    writer.writerow(df.columns)

    # Loop over each row and extract the LinkedIn link
    for index, row in df.iterrows():
        website = row['Website']
        if pd.notnull(website):
            linkedin_links = find_linkedin_link(website)
            if linkedin_links:
                df.at[index, 'LinkedIn_link'] = linkedin_links[0]

        # Print the row and write it to the output CSV file
        print(row)
        writer.writerow(row)

# Save the updated data to the new CSV file
df.to_csv('exhibitors_with_linkedin.csv', index=False, mode='a', header=False)

Name                                               3V Sigma S.p.a.
URL              https://www.in-cosmetics.com/global/en-gb/exhi...
Website                                 https://www.3vsigma.com/en
Email                                  fabio.costiniti@3vsigma.com
Phone                                                +390354165111
LinkedIn_link                                                     
Name: 0, dtype: object
Name                                           A&A Fratelli Parodi
URL              https://www.in-cosmetics.com/global/en-gb/exhi...
Website                               http://www.fratelliparodi.it
Email                                                          NaN
Phone                                              +39 010 792 151
LinkedIn_link                                                     
Name: 1, dtype: object
Name                                A&Z Science and Technology Ltd
URL              https://www.in-cosmetics.com/global/en-gb/exhi...
Website         

KeyboardInterrupt: 

In [6]:
# Read the CSV file
with open('exhibitors_full.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    header.append('LinkedIn_link')  # Add the new column name

    # Configure Chrome options to launch the driver in a minimized state
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    chrome_options.add_argument("--window-size=1920x1080")  # Set window size

    # Open the output file in append mode
    with open('exhibitors_with_linkedin.csv', 'a', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(header)  # Write the updated header row

        # Loop over each row and extract the LinkedIn link
        for row in reader:
            website = row[2]  # Assuming the 'Website' column is at index 2
            if website:
                linkedin_links = find_linkedin_link(website)
                if linkedin_links:
                    row.append(linkedin_links[0])  # Add the LinkedIn link to the row
            writer.writerow(row)  # Write the updated row to the output file

KeyboardInterrupt: 

In [ ]:
import csv
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def find_linkedin_company_people(secrets):
    driver = webdriver.Chrome()

    # Login
    driver.get("https://www.linkedin.com/login")
    time.sleep(randint(1, 5))
    username = driver.find_element(by=By.ID, value="username")
    username.send_keys(secrets["username"])
    password = driver.find_element(by=By.ID, value="password")
    password.send_keys(secrets["password"])
    login_button = driver.find_element(by=By.XPATH, value='//div[@id="organic-div"]/form/div[3]/button')
    login_button.click()
    time.sleep(randint(1, 5))

    # Open the existing CSV file for reading
    with open('exhibitors_with_linkedin.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        rows = list(reader)

    # Create a new list to store the updated rows
    updated_rows = []

    # Loop over the rows and process the LinkedIn links
    for row in rows:
        company_link = row['LinkedIn']
        if company_link:
            try:
                driver.get(f"{company_link}/people/")
                time.sleep(randint(1, 5))
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(randint(1, 5))
                    try:
                        driver.find_element(by=By.CLASS_NAME, value="artdeco-loader__bars")
                    except NoSuchElementException:
                        break
                time.sleep(randint(1, 5))
                html_content = driver.page_source
                with open(f"htmls/{company_link.split('/')[-1]}.html", "w", encoding='utf-8') as file:
                    file.write(html_content)
                row['Result'] = 'Success'
            except NoSuchElementException:
                time.sleep(randint(1, 5))
                row['Result'] = 'Failed'
            except Exception as e:
                print(f"Error processing {company_link}: {e}")
                row['Result'] = 'Error'
        else:
            row['Result'] = ''
        updated_rows.append(row)

    driver.quit()

    # Open a new CSV file for writing
    with open('exhibitors_with_linkedin_result.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Name', 'URL', 'Website', 'Email', 'Phone', 'LinkedIn', 'Result']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write the updated rows to the new CSV file
        writer.writerows(updated_rows)

    print('All exhibitors processed successfully with LinkedIn result.')

# Call the function with your secrets
secrets = {
    "username": "leticehoquetis@yahoo.com",
    "password": "L3tice.com"
}
find_linkedin_company_people(secrets)